# Universidade Tecnológica Federal do Paraná

## Programa de Pós-Graduação em Computação Aplicada

### Ciência de Dados 2 - 2021/1
### Equipe Evolution:
### Leila Fabiola Ferreira
### Mateus Cichelero da Silva

## Atividade 2 - Análise exploratória: Arquivo B - Formatação Dataset

Após a etapa de extração de dados, o próximo objetivo é formatar o dataset em uma estrutura que possibilite o fácil acesso às mensagens recebidas pelo bot. 

Conforme visto anteriormente, a base final de extração teve que ser dividida em vários arquivos menores para evitar problemas de memória. Aqui, iremos tratar essas colunas para separação das informações realmente necessárias, com o objetivo de tornar o arquivo final dataset.csv mais leve para os futuros processamentos.

In [84]:
import pandas as pd
from pandas.io.json import json_normalize
import json
from datetime import datetime

## Separando colunas de "raw text" do formato original de extração - Teste das funções em amostra de um dos arquivos extraídos

In [4]:
df = pd.read_csv('laura_extraction_data/0_data.csv')

In [5]:
df.shape

(10000, 3)

In [6]:
df.head()

,id,date_timestamp,data
0,5,1.585158e+09,"{""event"": ""user"", ""timestamp"": 1585157911.3968..."
1,13,1.585158e+09,"{""event"": ""user"", ""timestamp"": 1585157912.9197..."
2,17,1.585159e+09,"{""event"": ""user"", ""timestamp"": 1585158867.2471..."
3,25,1.585159e+09,"{""event"": ""user"", ""timestamp"": 1585158870.7418..."
4,29,1.585159e+09,"{""event"": ""user"", ""timestamp"": 1585158874.7780..."


Verificando amostra do campo "data"

In [69]:
json.loads(df["data"].iloc[0])

{'event': 'user',
 'timestamp': 1585157911.3968487,
 'text': 'Oi',
 'parse_data': {'intent': {'name': 'greet', 'confidence': 0.9729991555213928},
  'entities': [],
  'intent_ranking': [{'name': 'greet', 'confidence': 0.9729991555213928},
   {'name': 'menu_cure', 'confidence': 0.01178001705557108},
   {'name': 'doubts_coronavirus_confirmation',
    'confidence': 0.002744596917182207},
   {'name': 'doubts_no_symptoms_test', 'confidence': 0.002335932804271579},
   {'name': 'menu_covid', 'confidence': 0.002097305841743946},
   {'name': 'doubts_pandemic', 'confidence': 0.0019062263891100883},
   {'name': 'deny', 'confidence': 0.0018673930317163467},
   {'name': 'doubts_fever', 'confidence': 0.0017342010978609324},
   {'name': 'menu_transmission', 'confidence': 0.0012740789679810405},
   {'name': 'affirm', 'confidence': 0.0012611534912139177}],
  'text': 'Oi'},
 'input_channel': 'rest',
 'message_id': '9bd6cd59ad634214a04c67fda9ab654d',
 'metadata': {}}

In [107]:
#Exemplo aleatório de texto de mensagem
text[9100]

'Tem algum remédio que eu posso tomar?'

Aqui, nos interessa os campos *text*, relacionado a mensagem enviada, e *intent*, intenção classificada pelo chatbot.

In [80]:
text = df["data"].apply(lambda row:  json.loads(str(row))["text"])
intent = df["data"].apply(lambda row:  json.loads(str(row))["parse_data"]["intent"]["name"])

In [81]:
text.head()

0                     Oi
1          Tirar dúvidas
2                     Oi
3          Tirar dúvidas
4    Como é transmitido?
Name: data, dtype: object

In [82]:
intent.head()

0            greet
1           doubts
2            greet
3           doubts
4    doubts_spread
Name: data, dtype: object

Último passo para formatar o dataset na estrutura deseja é converter a coluna de timestamps para data:

In [94]:
date = pd.to_datetime(df["date_timestamp"],unit='s')

In [95]:
date.head()

0   2020-03-25 17:38:31.396850109
1   2020-03-25 17:38:32.919719934
2   2020-03-25 17:54:27.247100115
3   2020-03-25 17:54:30.741859913
4   2020-03-25 17:54:34.778069973
Name: date_timestamp, dtype: datetime64[ns]

## Generalizando para todo o dataset - Percorrendo todos os arquivos salvos e agregando o resultado em um único arquivo

In [97]:
df.to_csv("laura_processed_data/teste.csv", mode="a", header=True, index=False)

In [101]:
max_files = 126

final_header = ['id', 'date', 'text', 'intent']
    
for index_file in range(max_files):
    df = pd.read_csv('laura_extraction_data/'+str(index_file)+'_data.csv')
    
    #processamento e criação das colunas na estrutura final
    df['text'] = df["data"].apply(lambda row:  json.loads(str(row))["text"])
    df['intent'] = df["data"].apply(lambda row:  json.loads(str(row))["parse_data"]["intent"]["name"])
    df['date'] = pd.to_datetime(df["date_timestamp"],unit='s')
    
    #formatação na estrutura final do dataset
    df = df[final_header]
    
    #append no arquivo único final do dataset
    if index_file == 0 :
        df.to_csv("laura_processed_data/dataset.csv", header=True, index=False)
    else: 
        df.to_csv("laura_processed_data/dataset.csv", mode="a", header=False, index=False)

### Check do formato do dataset

In [103]:
df = pd.read_csv('laura_processed_data/dataset.csv')

In [104]:
df.shape

(1260000, 4)

O arquivo final conta com 1,26 milhão de mensagens, resultando em um csv de apenas 71MB, se comparado com os dados originais de extração, com seus 1,4GB.

In [105]:
df.head(15)

,id,date,text,intent
0,5,2020-03-25 17:38:31.396850109,Oi,greet
1,13,2020-03-25 17:38:32.919719934,Tirar dúvidas,doubts
2,17,2020-03-25 17:54:27.247100115,Oi,greet
3,25,2020-03-25 17:54:30.741859913,Tirar dúvidas,doubts
4,29,2020-03-25 17:54:34.778069973,Como é transmitido?,doubts_spread
5,33,2020-03-25 17:54:58.205969810,Oi,greet
6,37,2020-03-25 17:55:22.034669876,/menu_fake_news,menu_fake_news
7,41,2020-03-25 17:55:43.279109955,Oi,greet
8,49,2020-03-25 17:55:46.308470011,oi,greet
9,53,2020-03-25 17:55:48.087310076,Oi,greet


In [106]:
df.tail(15)

,id,date,text,intent
1259985,11012742,2021-03-20 01:15:51.118249893,Oi,greet
1259986,11012749,2021-03-20 01:16:02.618750095,Oi,greet
1259987,11012756,2021-03-20 01:16:04.145190001,Oi,greet
1259988,11012767,2021-03-20 01:16:14.324190140,Oi,greet
1259989,11012778,2021-03-20 01:16:26.922549963,Oi,greet
1259990,11012789,2021-03-20 01:16:41.087939978,Oi,greet
1259991,11012800,2021-03-20 00:28:59.777280092,Oi,greet
1259992,11012807,2021-03-20 00:29:40.066390038,Oi,greet
1259993,11012814,2021-03-20 00:30:04.072669983,sbc ja esta dando sobra de vacinas,doubts_vaccine_order
1259994,11012818,2021-03-20 00:30:10.899530172,/doubts_vaccine_order,doubts_vaccine_order
